## Groundwater Whirls with Particle Tracking (XT3D and PRT)

This is a 10 layer steady-state problem involving anisotropic groundwater
flow.  The XT3D formulation is used to represent anisotropy of the hydraulic
conductivity tensor.  The resulting flow pattern consists of groundwater whirls,
as described in the XT3D documentation report. A particle tracking model is
created to visualize the whirls.

### Initial setup

Import dependencies and define the example name and workspace.

In [1]:
import pathlib as pl
import warnings
from itertools import repeat

import flopy
import numpy as np
import pandas as pd
import pyvista as pv
from flopy.export.vtk import Vtk

warnings.simplefilter("ignore", UserWarning)

sim_name = "whirls-prt"
gwf_name = sim_name + "-gwf"
prt_name = sim_name + "-prt"
workspace = pl.Path("temp")
sim_ws = workspace / sim_name

# Define output file names
headfile_gwf = f"{gwf_name}.hds"
budgetfile_gwf = f"{gwf_name}.cbb"
budgetfile_prt = f"{prt_name}.cbb"
trackfile_prt = f"{prt_name}.trk"
trackhdrfile_prt = f"{prt_name}.trk.hdr"
trackcsvfile_prt = f"{prt_name}.trk.csv"

### Define parameters

Define model units, parameters and other settings.

In [2]:
# Model units
length_units = "meters"
time_units = "days"

# Model parameters
nper = 1  # Number of periods
nlay = 10  # Number of layers
nrow = 10  # Number of rows
ncol = 51  # Number of columns
delr = 100.0  # Spacing along rows ($m$)
delc = 100.0  # Spacing along columns ($m$)
top = 0.0  # Top of the model ($m$)
botm_str = "-100, -200, -300, -400, -500, -600, -700, -800, -900, -1000"  # Layer bottom elevations ($m$)
strt = 0.0  # Starting head ($m$)
icelltype = 0  # Cell conversion type
k11 = 1.0  # Hydraulic conductivity in the 11 direction ($m/d$)
k22 = 0.1  # Hydraulic conductivity in the 22 direction ($m/d$)
k33 = 1.0  # Hydraulic conductivity in the 33 direction ($m/d$)
angle1_str = "45, 45, 45, 45, 45, -45, -45, -45, -45, -45"  # Rotation of the hydraulic conductivity ellipsoid in the x-y plane
inflow_rate = 0.01  # Inflow rate ($m^3/d$)

# Static temporal data used by TDIS file
# Simulation has 1 steady stress period (1 day)
perlen = [1.0]
nstp = [1]
tsmult = [1.0]
tdis_ds = list(zip(perlen, nstp, tsmult))

# Parse strings into lists
botm = [float(value) for value in botm_str.split(",")]
angle1 = [float(value) for value in angle1_str.split(",")]

# Solver settings
nouter = 50
ninner = 100
hclose = 1e-9
rclose = 1e-6

### Model setup

Next we build models. In this example the groundwater flow (GWF) and particle tracking (PRT) model run within the same simulation.

In [3]:
sim = flopy.mf6.MFSimulation(sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6")

flopy.mf6.ModflowTdis(
    sim, nper=nper, perioddata=tdis_ds, time_units=time_units
)

gwf = flopy.mf6.ModflowGwf(sim, modelname=gwf_name, save_flows=True)

ims = flopy.mf6.ModflowIms(
    sim,
    linear_acceleration="bicgstab",
    outer_maximum=nouter,
    outer_dvclose=hclose,
    inner_maximum=ninner,
    inner_dvclose=hclose,
    rcloserecord=f"{rclose} strict",
)
sim.register_solution_package(ims, [gwf.name])

flopy.mf6.ModflowGwfdis(
    gwf,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)

flopy.mf6.ModflowGwfnpf(
    gwf,
    icelltype=icelltype,
    k=k11,
    k22=k22,
    k33=k33,
    angle1=angle1,
    save_specific_discharge=True,
    save_saturation=True,
    save_flows=True,
    xt3doptions=True,
)
flopy.mf6.ModflowGwfic(gwf, strt=strt)

rate = np.zeros((nlay, nrow, ncol), dtype=float)
rate[:, :, 0] = inflow_rate
rate[:, :, -1] = -inflow_rate
wellay, welrow, welcol = np.where(rate != 0.0)
wel_spd = [
    ((k, i, j), rate[k, i, j], 1 if rate[k, i, j] > 0 else 3)
    for k, i, j in zip(wellay, welrow, welcol)
]
wel_spd = {0: wel_spd}
flopy.mf6.ModflowGwfwel(
    gwf, stress_period_data=wel_spd, pname="WEL", auxiliary=["IFLOWFACE"]
)

flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=[headfile_gwf],
    budget_filerecord=[budgetfile_gwf],
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

prt = flopy.mf6.ModflowPrt(
    sim, modelname=prt_name, model_nam_file="{}.nam".format(prt_name)
)

flopy.mf6.ModflowGwfdis(
    prt,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)

flopy.mf6.ModflowPrtmip(prt, pname="mip", porosity=0.1)

xs = np.array(list(repeat(50, 4)))
ys = np.linspace(1, 999, 4)
zs = np.linspace(1, 999, 4) * -1
points = np.transpose(np.array(np.meshgrid(xs, ys, zs)).reshape(3, -1))
releasepts = [
    (i, *prt.modelgrid.intersect(*p), *p) for i, p in enumerate(points)
]
flopy.mf6.ModflowPrtprp(
    prt,
    nreleasepts=len(releasepts),
    packagedata=releasepts,
    perioddata={0: ["FIRST"]},
    exit_solve_tolerance=1e-5,
    istopzone=2,
    extend_tracking=True,
)

tracktimes = list(range(0, 700000000, 5000000))
flopy.mf6.ModflowPrtoc(
    prt,
    pname="oc",
    budget_filerecord=[budgetfile_prt],
    track_filerecord=[trackfile_prt],
    trackcsv_filerecord=[trackcsvfile_prt],
    saverecord=[("BUDGET", "ALL")],
    ntracktimes=len(tracktimes),
    tracktimes=[(t,) for t in tracktimes],
    track_release=True,
    track_terminate=True,
    track_usertime=True,
)

flopy.mf6.ModflowGwfprt(
    sim, exgtype="GWF6-PRT6", exgmnamea=gwf_name, exgmnameb=prt_name
)

ems = flopy.mf6.ModflowEms(
    sim,
    pname="ems",
    filename="{}.ems".format(prt_name),
)
sim.register_solution_package(ems, [prt.name])

### Run models

We are ready to run the models. First write model input files, then run the simulation.

In [4]:
# Write and run the simulation.
sim.write_simulation(silent=False)
success, buff = sim.run_simulation(silent=True)

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing solution package ems...
  writing package whirls-prt.gwfprt...
  writing model whirls-prt-gwf...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package ic...
    writing package wel...
INFORMATION: maxbound in ('gwf6', 'wel', 'dimensions') changed to 200 based on size of stress_period_data
    writing package oc...
  writing model whirls-prt-prt...
    writing model name file...
    writing package dis...
    writing package mip...
    writing package prp_0...
    writing package oc...


### Animate results

We can now load pathlines and create an animation.

First, load the pathlines from the CSV output file.

In [5]:
pls = pd.read_csv(sim_ws / trackcsvfile_prt)
pls

,kper,kstp,imdl,iprp,irpt,ilay,icell,izone,istatus,ireason,trelease,t,x,y,z,name
0,1,1,2,1,1,1,460,0,1,0,0.0,0.000000e+00,50.000000,1.000000,-1.000000,NaN
1,1,1,2,1,1,1,460,0,1,5,0.0,0.000000e+00,50.000000,1.000000,-1.000000,NaN
2,1,1,2,1,1,1,460,0,1,5,0.0,5.000000e+06,93.356684,1.226534,-0.894099,NaN
3,1,1,2,1,1,1,461,0,1,5,0.0,1.000000e+07,133.572578,1.461425,-0.789604,NaN
4,1,1,2,1,1,1,461,0,1,5,0.0,1.500000e+07,172.071885,1.731594,-0.695662,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8027,1,1,2,1,64,7,3569,0,1,5,0.0,6.750000e+08,4937.667714,0.003029,-669.464776,NaN
8028,1,1,2,1,64,7,3569,0,1,5,0.0,6.800000e+08,4984.222429,0.002298,-608.611738,NaN
8029,1,1,2,1,64,6,3060,0,1,5,0.0,6.850000e+08,5039.599246,0.001518,-523.445039,NaN
8030,1,1,2,1,64,5,2550,0,1,5,0.0,6.900000e+08,5091.144947,0.001811,-415.456719,NaN


Set some PyVista settings.

In [6]:
# pyvista settings
pv.set_jupyter_backend("static")
pv.set_plot_theme("document")
pv.global_theme.allow_empty_mesh = True

Create meshes for the model grid and the particle pathlines. We do this by exporting the the flow model and pathlines to VTK, then using the `to_pyvista()` conversion utility.

In [7]:
gwf = sim.get_model(gwf_name)
axes = pv.Axes(show_actor=False, actor_scale=2.0, line_width=5)
vtk = Vtk(model=gwf, binary=False, vertical_exageration=1, smooth=False)
vtk.add_model(gwf)
vtk.add_pathline_points(pls)
gwf_mesh, prt_mesh = vtk.to_pyvista()

Create another mesh for the wells. We do this by selecting the subset of the grid mesh corresponding to the well cells. This is possible because the mesh cell numbering is identical to the model grid node numbering.

In [8]:
wel_nodes = gwf.modelgrid.get_node([w[0] for w in wel_spd[0]])
wel_mesh = gwf_mesh.remove_cells(
    list(set(range(gwf.modelgrid.nnodes)) - set(wel_nodes)), inplace=False
)

We can create an animation of particles moving along their pathlines. First, slice the pathline mesh along the x axis, one slice per time point.

In [9]:
pathline_slices = prt_mesh.slice_along_axis(n=len(tracktimes), axis="x")

Now set up the plot and loop over the slices, saving each frame to a GIF file as we go.

In [10]:
# create the plotter
p = pv.Plotter(
    window_size=[700, 700],
    notebook=False,
    off_screen=True,
)
p.enable_anti_aliasing()

# add fixed meshes
p.add_mesh(gwf_mesh, opacity=0.1, style="wireframe")
p.add_mesh(wel_mesh, opacity=0.1, color="red", label="Wells")

# open a GIF file
p.open_gif("fixed_view.gif")
p.show(auto_close=False)

# add a mesh with the initial particle positions
frame = pathline_slices.get(0)
p.add_mesh(frame, point_size=12, color="black")

# update the particle mesh in-place for each timestamp,
# and write a new frame
for i, t in enumerate(tracktimes[1:]):
    frame.points = pathline_slices.get(i).points
    p.write_frame()

p.close()

Next we can modify the animation to take a particle's perspective as it travels through the model grid.

In [11]:
# create a new plotter
p = pv.Plotter(
    window_size=[700, 700],
    notebook=False,
    off_screen=True,
)
p.enable_anti_aliasing()

# add the fixed meshes
p.add_mesh(gwf_mesh, opacity=0.1, style="wireframe")
p.add_mesh(wel_mesh, opacity=0.1, color="red", label="Wells")

# open a new GIF file
p.open_gif("whirlycoaster.gif")
p.show(auto_close=False)

# this time we move the camera along with one of
# the particles. also add an overlay showing the
# current time and the particle's position. also
# we keep previous particle positions instead of
# overwriting them.
ntimes = len(tracktimes)
for i, t in enumerate(tracktimes[1:]):
    frame = pathline_slices.get(i)
    p.add_mesh(frame, point_size=12, color="black")
    x, y, z = frame.points[0]
    p.camera.position = (x, y, z)
    years = t / 365
    text = "\n".join(
        [
            f"Time: {years:.1f}y",
            f"X: {x:.0f}",
            f"Y: {y:.0f}",
            f"Z: {z:.0f}",
        ]
    )
    p.add_text(
        text,
        position="upper_left",
        font_size=18,
        color="black",
        name="time_text",
        shadow=True,
    )
    p.write_frame()

p.close()